In [214]:
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from dataWrangler import *
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from sklearn.linear_model import LinearRegression
import pandas as pd
from scrape_craigslist import Scrape
from predictions import *
import urllib.request
import warnings
warnings.filterwarnings('ignore')
import requests
import datetime
import math
import glob   
from xgboost import XGBRegressor
import os
import io
import re
import numpy as np

In [342]:
li=[]
for filename in glob.glob(r"..\UsedCar-PriceEstimator\data\*.csv"):
    collection = pd.read_csv(filename, index_col=None, header=0)
    li.append(collection)
frame = pd.concat(li, axis=0, ignore_index=True)


In [343]:
df=Replace(Spell(frame))

In [344]:
inputcar='toyota corolla'

In [345]:
df_make=Process(df,inputcar)

In [346]:
df1=Outliers_light(df_make)
df=delete_outliers(df1)

In [347]:
df.shape

(1104, 4)

In [348]:
df['score:'] = df['score:'].astype(int)
df['score:'] = df['score:'].astype(str)
df=pd.get_dummies(df)

In [349]:
df.shape

(1104, 7)

In [350]:
df.head()

,year:,odometer:,price:,score:_0,score:_1,score:_2,score:_3
0,16.0,209000.0,10.965784,0,0,1,0
1,6.0,36000.0,13.688250,0,0,1,0
2,20.0,91000.0,10.891784,0,1,0,0
3,15.0,139000.0,12.135709,0,0,0,1
4,26.0,162158.0,9.965784,0,0,1,0


### xgboost

In [359]:
X=df.drop(columns='price:')
y=df['price:']

train_features, test_features, train_labels, test_labels = \
train_test_split(X, y, test_size = 0.25, random_state = 30)


scaler = StandardScaler()
scaler.fit(train_features)



xgb =  XGBRegressor(learning_rate=0.1,gamma=0.5,max_depth=11,gamma=0.5,
                  eval_metric='mlogloss',n_estimators=100, eta=0.6, 
                   subsample=0.2, colsample_bytree=0.8)
xgb.fit(train_features, train_labels);

scaler.transform(test_features)

# Calculate mean absolute percentage error (MAPE)
y_pred=xgb.predict(test_features)
errors=np.abs((test_labels - y_pred))
mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)


print(f'{inputcar} Accuracy:', round(accuracy, 2), '%.')



SyntaxError: keyword argument repeated (<ipython-input-359-9045d721b13c>, line 13)

In [352]:
from sklearn.model_selection import RandomizedSearchCV


xgb_param_grid = {'gamma':[0, 0.01,0.1,0.3,0.5,0.8,1],
              'max_depth' : [3,5,7,9,11,13,15],
               'learning_rate': [0.1,0.2,0.4,0.7,0.9],
                'n_estimators' :[10,20,40,50,70,85,100,120]
             }

model = XGBRegressor(
                    objective='reg:linear', 
                    subsample=1, 
                    scale_pos_weight=1, 
                    seed=2866) 

random_search = RandomizedSearchCV(model, 
                               param_distributions=xgb_param_grid,
                               n_iter=4,
                               verbose=2)
fitted_model=random_search.fit(train_features, train_labels)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8 ......
[13:11:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8, total=   0.3s
[CV] n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8 ......
[13:11:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8, total=   0.2s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s



[CV] n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8 ......
[13:11:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8, total=   0.2s
[CV] n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8 ......
[13:11:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8, total=   0.2s
[CV] n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8 ......
[13:11:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimators=50, max_depth=9, learning_rate=0.7, gamma=0.8, total=   0.2s
[CV] n_estimato

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    5.0s finished


In [353]:
fitted_model.best_params_

{'n_estimators': 100, 'max_depth': 13, 'learning_rate': 0.1, 'gamma': 0.5}

### random forest

In [358]:
X=df.drop(columns='price:')
y=df['price:']

train_features, test_features, train_labels, test_labels = \
train_test_split(X, y, test_size = 0.25, random_state = 55)


scaler = StandardScaler()
scaler.fit(train_features)


rf1 = RandomForestRegressor(n_estimators=94,
                            min_samples_split=10,
                            max_depth=70)
rf1.fit(train_features, train_labels);

scaler.transform(test_features)

# Calculate mean absolute percentage error (MAPE)
y_pred=rf1.predict(test_features)
errors=np.abs((test_labels - y_pred))
mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)
print(f'{inputcar} Accuracy:', round(accuracy, 2), '%.')

toyota corolla Accuracy: 98.0 %.


In [355]:

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 10)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]# Create the random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestRegressor()


rf_random = RandomizedSearchCV(rf, param_distributions = random_grid,
                               n_iter = 8, cv = 5,
                               random_state=42)

fitted_randomforest=rf_random.fit(train_features, train_labels)



In [356]:
fitted_randomforest.best_params_

{'n_estimators': 94,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': False}

### linear regression

In [357]:
X=df.drop(columns='price:')
y=df['price:']

train_features, test_features, train_labels, test_labels = \
train_test_split(X, y, test_size = 0.25, random_state = 32)


scaler = StandardScaler()
scaler.fit(train_features)


reg = LinearRegression()
reg.fit(train_features, train_labels);


# Calculate mean absolute percentage error (MAPE)
y_pred=reg.predict(test_features)
errors=np.abs((test_labels - y_pred))
mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)
print(f'{inputcar} Accuracy:', round(accuracy, 2), '%.')

toyota corolla Accuracy: 97.85 %.
